## Vision por Computador: Práctica 6

# Juego razas

### Variables e importaciones necesarias

In [115]:
# Importa los modulos necesarios
import cv2
import os
from deepface import DeepFace

# Cargamos los gorros guardados
gorros = {
        'middle eastern': cv2.imread('assets/gorro_ruso.png', cv2.IMREAD_UNCHANGED),
        'asian': cv2.imread('assets/gorro_chino.png', cv2.IMREAD_UNCHANGED),
        'indian': cv2.imread('assets/gorro_indio.png', cv2.IMREAD_UNCHANGED),
        'latino hispanic': cv2.imread('assets/gorro_mexicano.png', cv2.IMREAD_UNCHANGED),
        'black': cv2.imread('assets/gorro_negro.png', cv2.IMREAD_UNCHANGED),
        'white': cv2.imread('assets/gorro_blanco.png', cv2.IMREAD_UNCHANGED),
        'default': cv2.imread('assets/gorro_blanco.png', cv2.IMREAD_UNCHANGED)
        }

for nombre, gorro in gorros.items():
    print(f"{nombre}: {gorro.shape}")  # Debería mostrar algo como (altura, anchura, 4)

class DynamicButton:
    def __init__(self, x1, y1, width, height, res_width, res_height):
        # Calcular las proporciones en función de la resolución de referencia (900x699)
        ref_width, ref_height = 900, 699
        self.width = width * res_width / ref_width
        self.height = height * res_height / ref_height
        self.x1 = x1 * res_width / ref_width
        self.y1 = y1 * res_height / ref_height

    def get_coordinates(self):
        x2 = self.x1 + self.width
        y2 = self.y1 + self.height
        return (int(self.x1), int(self.y1), int(x2), int(y2))

class DynamicButtons:
    def __init__(self, res_width, res_height):
        self.MENU = {
            "ELEGIR": DynamicButton(50, 285, 245, 105, res_width, res_height).get_coordinates(),
            "DETECTAR": DynamicButton(325, 285, 245, 105, res_width, res_height).get_coordinates(),
            "SALIR": DynamicButton(600, 285, 245, 105, res_width, res_height).get_coordinates(),
        }

        self.DETECTAR = {
            "MENU": DynamicButton(35, 580, 160, 80, res_width, res_height).get_coordinates(),
            "SALIR": DynamicButton(710, 580, 160, 80, res_width, res_height).get_coordinates()
        }

        self.ELEGIR = {
            "MENU": DynamicButton(35, 15, 160, 80, res_width, res_height).get_coordinates(),
            "SALIR": DynamicButton(710, 15, 160, 80, res_width, res_height).get_coordinates()
        }

        self.CAMBIAR_SOMBRERO = {
            'black': DynamicButton(55, 545, 110, 110, res_width, res_height).get_coordinates(),
            'indian': DynamicButton(190, 545, 110, 110, res_width, res_height).get_coordinates(),
            'asian': DynamicButton(325, 545, 110, 110, res_width, res_height).get_coordinates(),
            'latino hispanic': DynamicButton(460, 545, 110, 110, res_width, res_height).get_coordinates(),
            'middle eastern': DynamicButton(595, 545, 110, 110, res_width, res_height).get_coordinates(),
            'white': DynamicButton(735, 545, 110, 110, res_width, res_height).get_coordinates(),
        }

'''
# Ejemplo de uso
vid = cv2.VideoCapture('./video_test_5.mp4')

# Obtiene el tamaño del fotograma
width = int(vid.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(vid.get(cv2.CAP_PROP_FRAME_HEIGHT))

# Imprime el tamaño del fotograma inicial
print(f"Tamaño del fotograma : {width}x{height}")

# Libera el objeto de captura
vid.release()

# Crea los botones dinámicos
dynamic_buttons = DynamicButtons(width, height)  
print(dynamic_buttons.botones_menu)
'''

class GameInterface:
    def __init__(self, res_width, res_height):
        self.res_width = res_width
        self.res_height = res_height
        self.MENU = self.cargar_y_redimensionar('assets/pantalla_menu.png')
        self.DETECTAR = self.cargar_y_redimensionar('assets/pantalla_detectar.png')
        self.ELEGIR = {
            'white': self.cargar_y_redimensionar('assets/GorroBlancoSeleccionado.png'),
            'indian': self.cargar_y_redimensionar('assets/GorroIndioSeleccionado.png'),
            'asian': self.cargar_y_redimensionar('assets/GorroChinoSeleccionado.png'),
            'latino hispanic': self.cargar_y_redimensionar('assets/GorroMexicanoSeleccionado.png'),
            'black': self.cargar_y_redimensionar('assets/GorroNegroSeleccionado.png'),
            'middle eastern': self.cargar_y_redimensionar('assets/GorroRusoSeleccionado.png')
        }

    def cargar_y_redimensionar(self, ruta_imagen):
        imagen = cv2.imread(ruta_imagen, cv2.IMREAD_UNCHANGED)
        imagen_redimensionada = cv2.resize(imagen, (self.res_width, self.res_height))
        return imagen_redimensionada

# Ejemplo de uso
# game_interface = GameInterface(1280, 720)

class GameState:
    def __init__(self):
        self.current_state = "MENU" # Estados posibles: "MENU", "DETECTAR", "ELEGIR", "SALIR"
        self.selected_hat = 'white'

    def change_state(self, new_state):
        self.current_state = new_state

    def set_selected_hat(self, hat):
        self.selected_hat = hat



# Estado del juego
# estado = "MENU"  # Estados posibles: "MENU", "DETECTAR", "ELEGIR", "SALIR"
# Diccionario para almacenar el gorro seleccionado
# gorro_seleccionado = 'white'


middle eastern: (612, 612, 4)
asian: (512, 512, 4)
indian: (724, 600, 4)
latino hispanic: (1790, 1920, 4)
black: (1595, 1920, 4)
white: (375, 385, 4)
default: (375, 385, 4)


In [116]:
def superponer_imagen(fondo, superposicion):
    
    superposicion = cv2.resize(superposicion, (fondo.shape[1], fondo.shape[0]))
    # Extraer el canal alpha de la superposición
    alpha_s = superposicion[:, :, 3] / 255.0
    alpha_l = 1 - alpha_s

    # Superponer la imagen
    for c in range(0, 3):
        fondo[:, :, c] = (alpha_s * superposicion[:, :, c] +
                          alpha_l * fondo[:, :, c])
        
def superponer_gorro(frame, gorro, x, y, w, h):
    gorro_redimensionado = cv2.resize(gorro, (w, int(h / 2)))
    for i in range(gorro_redimensionado.shape[0]):
        for j in range(gorro_redimensionado.shape[1]):
            if gorro_redimensionado[i, j, 3] != 0:
                frame[y - int(h / 2) + i, x + j, :] = gorro_redimensionado[i, j, 0:3]
    return frame

def __detectar(frame):
    # Variable estática para el conteo de frames
    if 'contador_frames' not in detectar.__dict__:
        detectar.contador_frames = 0  # Inicializar en la primera llamada
    if 'obj' not in detectar.__dict__:
        detectar.obj = DeepFace.analyze(img_path=frame, enforce_detection=False, actions=['race'])  # Almacenar los últimos resultados de DeepFace
        
    face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
    font, fontScale, color, thickness, pos = cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2, (50, 50)


    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(100, 100))

    # Incrementar contador de frames
    detectar.contador_frames += 1
    
    # Aplicar DeepFace solo cada 50 frames
    if detectar.contador_frames % 40 == 0 and len(faces) > 0:
        # Analizar todas las caras a la vez
        detectar.obj = DeepFace.analyze(img_path=frame, enforce_detection=False, actions=['race'])
        print(detectar.obj)
        #print(obj)

    for i, (x, y, w, h) in enumerate(faces):
        if i < len(detectar.obj):
            # Obtenemos la raza dominante para la cara actual
            raza_dominante = detectar.obj[i]["dominant_race"]
            gorro = gorros.get(raza_dominante, gorros['default'])
            
            frame = superponer_gorro(frame, gorro, x, y, w, h)
            #cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)
            #cv2.putText(frame, raza_dominante, pos, font, fontScale, color, thickness, cv2.LINE_AA)
        else:
            break  # No hay más resultados de DeepFace para procesar
        
    return frame

def detectar(frame):
    if 'contador_frames' not in detectar.__dict__:
        detectar.contador_frames = 0  # Inicializar en la primera llamada
    if 'obj' not in detectar.__dict__:
        detectar.obj = DeepFace.analyze(img_path=frame, enforce_detection=False, actions=['race'])  # Almacenar los últimos resultados de DeepFace
    
    # Incrementar contador de frames
    detectar.contador_frames += 1
    
    # Aplicar DeepFace solo cada 40 frames
    if detectar.contador_frames % 5 == 0:
        # Analizar todas las caras a la vez
        detectar.obj = DeepFace.analyze(img_path=frame, enforce_detection=False, actions=['race'])
        print(detectar.obj)

    if detectar.obj:
        for resultado in detectar.obj:
            confianza = resultado['face_confidence']
            if confianza >= 2:
                # Extraer la información de la región y la raza dominante
                region = resultado['region']
                raza_dominante = resultado['dominant_race']
    
                # Coordenadas de la cara
                x, y, w, h = region['x'], region['y'], region['w'], region['h']
    
                # Seleccionar el gorro basado en la raza dominante
                gorro = gorros.get(raza_dominante, gorros['default'])
    
                # Superponer el gorro en la cara detectada
                frame = superponer_gorro(frame, gorro, x, y, w, h)
                # Opcional: Dibujar rectángulo y texto
                # cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)
                # font, fontScale, color, thickness, pos = cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2, (50, 50)
                # cv2.putText(frame, raza_dominante, (x, y - 10), font, fontScale, color, thickness, cv2.LINE_AA)
        
    return frame




def elegir(frame, gorro_seleccionado):
    # global gorro_seleccionado
    face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=6, minSize=(150, 150))

    # Superponer el gorro seleccionado sobre cada cara detectada
    for (x, y, w, h) in faces:
        if gorro_seleccionado is not None:
            gorro = gorros[gorro_seleccionado]
            frame = superponer_gorro(frame, gorro, x, y, w, h)
            # cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)

    return frame

def salir():
    print("Saliendo del juego")
    cv2.destroyAllWindows()



                   


In [117]:
class Game:
    def __init__(self, source_video):
        self.vid = self.__check_source(source_video)
        self.game_interface = self.__init_component(GameInterface)
        self.dynamic_buttons =  self.__init_component(DynamicButtons)
        self.game_state = GameState()
        
    def __check_source(self, source):
        vid = cv2.VideoCapture(source)
        if not vid.isOpened():
            raise Exception(f"Error al abrir la fuente de video: {source}")
        return vid
    
    def __init_component(self, ComponentClass):
        width = int(self.vid.get(cv2.CAP_PROP_FRAME_WIDTH))
        height = int(self.vid.get(cv2.CAP_PROP_FRAME_HEIGHT))
        return ComponentClass(res_height=height, res_width=width)
        
    def start(self):
        
        cv2.namedWindow('Juego')
        cv2.setMouseCallback('Juego', self.clic_mouse, {'dynamic_buttons': self.dynamic_buttons})

        while True:
            ret, frame = self.vid.read()
            if not ret:
                print("No se pudo leer el frame. Fin del video o error de lectura.")
                break

            self.manage_game_state(frame)

            cv2.imshow('Juego', frame)
            
            if (cv2.waitKey(1) & 0xFF == 27) or self.game_state.current_state == "SALIR":
                break

        self.vid.release()
        cv2.destroyAllWindows()

    def exit(self):
        self.game_state.current_state = "SALIR"   
        
    def manage_game_state(self, frame):
        estado = self.game_state.current_state
        if estado == "MENU":
            self.dibujar_botones(frame, self.game_interface.MENU, self.dynamic_buttons.MENU)
        elif estado == "DETECTAR":
            frame = detectar(frame)
            self.dibujar_botones(frame, self.game_interface.DETECTAR, self.dynamic_buttons.DETECTAR)
        elif estado == "ELEGIR":
            frame = elegir(frame, self.game_state.selected_hat)
            self.dibujar_botones(frame, self.game_interface.ELEGIR[self.game_state.selected_hat], self.dynamic_buttons.ELEGIR)
       
        # No se necesita un caso para "SALIR", ya que se maneja en el bucle principal

    def dibujar_botones(self, frame, pantalla, botones):
        superponer_imagen(fondo=frame, superposicion=pantalla)
        for nombre_boton, (x1, y1, x2, y2) in botones.items():
            cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)

    def clic_mouse(self, event, x, y, flags, param):
        buttons = param['dynamic_buttons']
        if event == cv2.EVENT_LBUTTONDOWN:
            estado = self.game_state.current_state
            if estado == "MENU":
                self.actualizar_estado(x, y, buttons.MENU)
            elif estado == "DETECTAR":
                self.actualizar_estado(x, y, buttons.DETECTAR)
            elif estado == "ELEGIR":
                self.actualizar_estado(x, y, buttons.ELEGIR)
                self.actualizar_sombrero_seleccionado(x, y, buttons.CAMBIAR_SOMBRERO)

    def actualizar_estado(self, x, y, botones):
        for nombre_boton, (x1, y1, x2, y2) in botones.items():
            if x1 <= x <= x2 and y1 <= y <= y2:
                self.game_state.change_state(nombre_boton)
                break

    def actualizar_sombrero_seleccionado(self, x, y, botones):
        for nombre_gorro, (x1, y1, x2, y2) in botones.items():
            if x1 <= x <= x2 and y1 <= y <= y2:
                self.game_state.set_selected_hat(nombre_gorro)
                break
            

game = Game('./video_test_5.mp4')
game.start()


[{'race': {'asian': 39.7924077758303, 'indian': 2.346536051149839, 'black': 12.771026757833152, 'white': 29.662206213211274, 'middle eastern': 5.9480350824552675, 'latino hispanic': 9.4797823453209}, 'dominant_race': 'asian', 'region': {'x': 0, 'y': 0, 'w': 1280, 'h': 640}, 'face_confidence': 0}]
[{'race': {'asian': 10.371468834368518, 'indian': 25.46570750866577, 'black': 52.1413238132789, 'white': 1.7899564602945126, 'middle eastern': 2.8186289410966006, 'latino hispanic': 7.412916118676431}, 'dominant_race': 'black', 'region': {'x': 455, 'y': 112, 'w': 248, 'h': 248}, 'face_confidence': 6.390769102727063}]
[{'race': {'asian': 58.510092408066534, 'indian': 0.7043805393207395, 'black': 3.029914753431961, 'white': 28.937559064481203, 'middle eastern': 3.2501908093804603, 'latino hispanic': 5.567866150609621}, 'dominant_race': 'asian', 'region': {'x': 0, 'y': 0, 'w': 1280, 'h': 640}, 'face_confidence': 0}]
[{'race': {'asian': 15.867253736584933, 'indian': 14.613401637863918, 'black': 52